# Estrategia de Modelado Predictivo (ML)

**Proyecto:** Optimización de Eficiencia Energética Naval
**Fase:** Selección de Arquitectura y Entrenamiento (Model Tournament)

---

**Objetivo del Módulo**

Desarrollar y comparar modelos de Machine Learning para predecir el consumo de combustible (`log_fuel_consumption`), superando las limitaciones de linealidad detectadas en el Baseline Físico (OLS).

**Contexto Técnico (Heredado del EDA)**

El análisis exploratorio reveló tres factores críticos que definen la estrategia de modelado:
1.  **No-Linealidad Estructural:** La "Ley del Cubo" teórica se ve distorsionada a baja velocidad por la Carga Base (*Hotel Load*), creando un efecto de "banana" que la regresión lineal no captura bien.
2.  **Autocorrelación Severa:** El test de Durbin-Watson (0.01) confirmó que los datos dependen fuertemente del tiempo.
3.  **Multicolinealidad:** La variable `width` fue eliminada; el modelo debe manejar la interacción entre `length` y `draft` para deducir la resistencia.

**Estrategia de Validación (Rules of Engagement)**

Para garantizar resultados realistas y evitar *Data Leakage*:
* **Prohibido `shuffle=True`:** No mezclaremos datos aleatoriamente.
* **Split Cronológico:** Entrenaremos estrictamente con el **Pasado (80%)** y evaluaremos con el **Futuro (20%)**.
* **Métrica Decisiva:** RMSE (Root Mean Squared Error) para penalizar grandes desviaciones.

---
**Los Modelos a evaluar**

1.  **Baseline:** Regresión Lineal (OLS) - *Referencia de "suelo" de rendimiento.*
2.  **Retador 1:** Random Forest Regressor - *Captura de no-linealidad robusta.*
3.  **Retador 2:** XGBoost Regressor - *Boosting de alto rendimiento para Big Data.*

## Instalacion librerias y configuracion de Entorno

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# Métricas de Evaluación
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
# Herramientas de Validación (Series de Tiempo)
from sklearn.model_selection import TimeSeriesSplit
# Modelos
from sklearn.linear_model import LinearRegression   # Benchmark (Línea Base)
from sklearn.ensemble import RandomForestRegressor  # Retador 1 (No linealidad)
import xgboost as xgb                               # Retador 2 (Potencia Big Data)

# --- Configuración Global ---
# Estilo de gráficos limpio
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# Configuración de Pandas para ver todas las columnas
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.4f' % x)

# Silenciar advertencias molestas de versiones futuras
import warnings
warnings.filterwarnings('ignore')

print("✅ Entorno Configurado Exitosamente.")
print(f"   - Pandas Version: {pd.__version__}")
print(f"   - XGBoost Version: {xgb.__version__}")
print("   - Estrategia: TimeSeriesSplit cargado.")

✅ Entorno Configurado Exitosamente.
   - Pandas Version: 2.3.3
   - XGBoost Version: 3.1.2
   - Estrategia: TimeSeriesSplit cargado.


## Función de Evaluación Estandarizada

In [2]:
def train_evaluate_model(model, model_name, X_train, y_train, X_test, y_test):
    """ Entrena el modelo y evalúa su desempeño en train y test para detectar overfitting.
    
    Args:
        model: Estimador de scikit-learn o xgboost inicializado.
        model_name (str): Nombre identificador.
    
    Returns:
        dict: Métricas calculadas.
    """
    
    # 1. Entrenamiento
    print(f"🔄 Entrenando {model_name}...")
    model.fit(X_train, y_train)
    
    # 2. Predicción (Train y Test)
    # Predecimos en ambos para medir la brecha (Generalización vs Memorización)
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    
    # 3. Cálculo de Métricas
    # RMSE (Root Mean Squared Error): Penaliza cuadráticamente los errores grandes.
    rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
    rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
    
    # MAE (Mean Absolute Error): Error promedio directo (más interpretable).
    mae_test = mean_absolute_error(y_test, y_pred_test)
    
    # R2 (Coeficiente de Determinación): Capacidad explicativa de la varianza.
    r2_test = r2_score(y_test, y_pred_test)
    
    # 4. Reporte Técnico en Consola
    print(f"\n📊 Resultados para: {model_name}")
    print(f"{'-'*30}")
    print(f"• RMSE Train: {rmse_train:.4f}")
    print(f"• RMSE Test:  {rmse_test:.4f}")
    print(f"• Delta RMSE: {rmse_test - rmse_train:.4f} (Gap de Overfitting)")
    print(f"• MAE Test:   {mae_test:.4f}")
    print(f"• R² Test:    {r2_test:.4f}")
    print(f"{'='*30}\n")
    
    return {
        'Modelo': model_name,
        'RMSE_Test': rmse_test,
        'RMSE_Train': rmse_train,
        'MAE_Test': mae_test,
        'R2_Test': r2_test
    }

# Lista para guardar los resultados del torneo
results_list = []

print("✅ Función 'train_evaluate_model' definida. El sistema de arbitraje está listo.")

✅ Función 'train_evaluate_model' definida. El sistema de arbitraje está listo.


**Puntos Clave:**

1. Delta RMSE: Calculamos la diferencia entre el error de entrenamiento y el de prueba. Si este número es muy alto, el modelo está haciendo Overfitting (memorizando en vez de aprender).

2. Métrica Dual (RMSE vs MAE): RMSE es más sensible a los valores atípicos (picos de consumo), mientras que MAE nos da una idea del error "en el día a día".

In [ ]:
# --- CELDA 3: FASE 1 - Benchmark (Regresión Lineal OLS) ---

# 1. Inicializar el modelo base
# Usamos los parámetros por defecto. No requiere ajuste de hiperparámetros.
lr_model = LinearRegression()

# 2. Entrenar y Evaluar usando la función del árbitro
# Pasamos los datos de Train (Pasado) y Test (Futuro)
print("🏁 Iniciando Benchmark...")
lr_results = train_evaluate_model(
    model=lr_model,
    model_name="Baseline (Regresión Lineal)",
    X_train=X_train,
    y_train=y_train,
    X_test=X_test,
    y_test=y_test
)

# 3. Guardar resultados en el tablero del torneo
results_list.append(lr_results)